This notebook scrapes simbli websites and get the school district details.

Input
- `s_tokens.json`

In [40]:
import json

with open("s_tokens.json",'r') as f:
    s_tokens = json.load(f)

for key, value in s_tokens.items():
    s_tokens[key] = [item for item in value if item != '']
    if len(s_tokens[key]) < 3:
        s_tokens[key].append("")

In [41]:
len(s_tokens)

2016

In [42]:
s_tokens

{'2': ['Kirksville R-III School District',
  'MO',
  ['Kirksville R-III School District 1901 E Hamilton Street  Kirksville, MO 63501',
   'Phone: 660.665.7774',
   'Fax: 660.665.3281',
   'http://www.kirksville.k12.mo.us']],
 '4': ['North Andrew Co. R-VI',
  'MO',
  [None, None, None, 'https://northandrew.org/']],
 '6': ['Savannah R-III',
  'MO',
  ['Savannah R-III 408 W Market St  Savannah, MO 64485',
   'Phone: 816-324-3144',
   'Fax: 816-324-5594',
   'http://www.savannahr3.com']],
 '10': ['Alamance-Burlington School System',
  'NC',
  ['Alamance-Burlington School System 1712 Vaughn Road  Burlington, NC 27217',
   'Phone: 336-570-6060',
   None,
   'http://www.abss.k12.nc.us']],
 '11': ['Van-Far R-I', 'MO', [None, None, None, 'https://www.vf.k12.mo.us/']],
 '12': ['Mexico Public Schools #59',
  'MO',
  ['Mexico Public Schools #59 2101 Lakeview  Mexico, MO 65265',
   'Phone: (573) 581-3773',
   'Fax: (573) 581-1794',
   'http://www.mexicoschools.net/']],
 '14': ['Southwest R-V', 'MO'

In [43]:
import random

# Convert dictionary keys to a list before sampling
# subset_keys = list(s_tokens.keys())[1850:]
# 36030535

# Create the subset dictionary
# urls_subset = {key: s_tokens[key] for key in subset_keys}  # Use a new variable to avoid overwriting

urls_subset = {}

for key in s_tokens:
    if s_tokens[key][2][3] == None or (s_tokens[key][2][0] == None and s_tokens[key][2][1] == None and s_tokens[key][2][2] == None and s_tokens[key][2][3] == ""):
    # if s_tokens[key][2][3] == None:
        urls_subset[key] = s_tokens[key] 

urls_subset

{'436': ['Bayless', 'MO', [None, None, None, None]],
 '36030903': ['Barstow Unified School  District',
  'CA',
  [None, None, None, None]],
 '36030934': ['New Hanover County Schools', 'NC', [None, None, None, None]]}

In [44]:
len(urls_subset)

3

In [45]:
import http.client
from bs4 import BeautifulSoup
import re
import time

# Define the connection to the ScrapingAnt API
conn = http.client.HTTPSConnection("api.scrapingant.com")

count = 0

for key in urls_subset:
    if urls_subset[key][2][3] == "" or urls_subset[key][2][3] == None:
        address = None
        # Request data
        # conn.request("GET", "/v2/general?url=https%3A%2F%2Fsimbli.eboardsolutions.com%2FIndex.aspx%3FS%3D" + key + "&x-api-key=e28d8e139b6d4207be20be3be425870e")
        # conn.request("GET", "/v2/general?url=https%3A%2F%2Fsimbli.eboardsolutions.com%2FIndex.aspx%3FS%3D" + key + "&x-api-key=d041188143fa46509ebd4b7a488d8766")
        conn.request("GET", "/v2/general?url=https%3A%2F%2Fsimbli.eboardsolutions.com%2FIndex.aspx%3FS%3D" + key + "&x-api-key=6a352d30dd76418da74b0d423f5a56bd")

        res = conn.getresponse()

        # Read the response
        data = res.read()

        # Decode the HTML content
        html_content = data.decode("utf-8")

        # Parse the HTML with BeautifulSoup
        soup = BeautifulSoup(html_content, "html.parser")
        count += 1
        print("Done " + str(count))

        # Find the address, phone, fax, and website
        address_element = soup.find("address")
        phone_number = None
        fax_number = None

        website_element = soup.find("a", class_="dv-logo")
        # print(website_element)
        website = website_element["href"] if website_element else None

        if address_element:
            # Find the address, phone, fax, and website from the address block
            address_lines = address_element.find_all("span", class_="sa-details")

            # Assuming that address details are in the right order in the sa-details span
            address = " ".join([line.get_text().strip() for line in address_lines[:4]])  # First four spans
            phone_number = address_lines[4].get_text().strip() if len(address_lines) > 4 else None
            fax_number = address_lines[5].get_text().strip() if len(address_lines) > 5 else None
            website = address_element.find("a", class_="websiteUrl")["href"] if address_element.find("a", class_="websiteUrl") else None

        # Output the results
        # print("Address:", address)
        # print("Phone Number:", phone_number)
        # print("Fax Number:", fax_number)
        # print("Website:", website)
        urls_subset[key][2] = [address, phone_number, fax_number, website]
        # time.sleep(2)
        
        if (count%10 == 0):
            for key in urls_subset:
                if s_tokens[key][2][3] == None or (s_tokens[key][2][0] == None and s_tokens[key][2][1] == None and s_tokens[key][2][2] == None and s_tokens[key][2][3] == ""):
                    s_tokens[key][2] = urls_subset[key][2]

            with open('s_tokens.json', 'w') as file:
                json.dump(s_tokens, file, indent=4)
            

urls_subset


Done 1


KeyboardInterrupt: 

In [46]:
urls_subset

{'436': ['Bayless', 'MO', [None, None, None, None]],
 '36030903': ['Barstow Unified School  District',
  'CA',
  [None, None, None, None]],
 '36030934': ['New Hanover County Schools', 'NC', [None, None, None, None]]}